In [1]:
!pip install cvxopt

In [0]:
import cvxopt.glpk
import cvxopt as cv
import numpy as np
import time
np.set_printoptions(threshold=10)


## Peg Solitaire, with the formalism of Masashi KIYOMI and Tomomi MATSUI

We will use the formalism from the paper by Masashi KIYOMI and Tomomi MATSUI.

A position of the pegs on the board will be represented by a dimension n = 33 vector.

We will build the matrix *moves* (called *A* in the paper), which contains al possible moves.

*aij* is :
* 1 iff the peg i is removed by the move j
* -1 iff it is added
* 0 in other cases.





In [0]:
#here we build moves
A = np.zeros((33, 76))
A[0][0] = 1
A[3][0] = 1
A[8][0] = -1

A[1][1] = 1
A[4][1] = 1
A[9][1] = -1

A[2][2] = 1
A[5][2] = 1
A[10][2] = -1

A[3][3] = 1
A[8][3] = 1
A[15][3] = -1

A[4][4] = 1
A[9][4] = 1
A[16][4] = -1

A[5][5] = 1
A[10][5] = 1
A[17][5] = -1

A[22][13] = 1
A[27][13] = 1
A[30][13] = -1

A[23][14] = 1
A[28][14] = 1
A[31][14] = -1

A[24][15] = 1
A[29][15] = 1
A[32][15] = -1

A[15][16] = 1
A[22][16] = 1
A[27][16] = -1

A[16][17] = 1
A[23][17] = 1
A[28][17] = -1

A[17][18] = 1
A[24][18] = 1
A[29][18] = -1

for j in range(7):
  A[6 + j][6 + j] = 1
  A[13 + j][6 + j] = 1
  A[20+j][6+j] = -1

  
A[0][34] = 1
A[1][34] = 1
A[2][34] = -1

A[3][35] = 1
A[4][35] = 1
A[5][35] = -1

A[27][36] = 1
A[28][36] = 1
A[29][36] = -1

A[30][37] = 1
A[31][37] = 1
A[32][37] = -1

for j in range(5):
  A[6 + j][19 + j] = 1
  A[7 + j][19 + j] = 1
  A[8+j][19+j] = -1
for j in range(5):
  A[13 + j][24 + j] = 1
  A[14 + j][24 + j] = 1
  A[15+j][24+j] = -1
for j in range(5):
  A[20 + j][29 + j] = 1
  A[21 + j][29 + j] = 1
  A[22+j][29+j] = -1
  

A[0][0+38] = -1
A[3][0+38] = 1
A[8][0+38] = 1

A[1][1+38] = -1
A[4][1+38] = 1
A[9][1+38] = 1

A[2][2+38] = -1
A[5][2+38] = 1
A[10][2+38] =1

A[3][3+38] = -1
A[8][3+38] = 1
A[15][3+38] = 1

A[4][4+38] = -1
A[9][4+38] = 1
A[16][4+38] = 1

A[5][5+38] = -1
A[10][5+38] = 1
A[17][5+38] = 1

A[22][13+38] = -1
A[27][13+38] = 1
A[30][13+38] = 1

A[23][14+38] = -1
A[28][14+38] = 1
A[31][14+38] = 1

A[24][15+38] = -1
A[29][15+38] = 1
A[32][15+38] = 1

A[15][16+38] = -1
A[22][16+38] = 1
A[27][16+38] = 1

A[16][17+38] = -1
A[23][17+38] = 1
A[28][17+38] = 1

A[17][18+38] = -1
A[24][18+38] = 1
A[29][18+38] = 1

for j in range(7):
  A[6 + j][6 + j+38] = -1
  A[13 + j][6 + j+38] = 1
  A[20+j][6+j+38] = 1

  
A[0][34+38] = -1
A[1][34+38] = 1
A[2][34+38] = 1

A[3][35+38] = -1
A[4][35+38] = 1
A[5][35+38] = 1

A[27][36+38] = -1
A[28][36+38] = 1
A[29][36+38] = 1

A[30][37+38] = -1
A[31][37+38] = 1
A[32][37+38] = 1

for j in range(5):
  A[6 + j][19 + j+38] = -1
  A[7 + j][19 + j+38] = 1
  A[8+j][19+j+38] = 1
for j in range(5):
  A[13 + j][24 + j+38] = -1
  A[14 + j][24 + j+38] = 1
  A[15+j][24+j+38] = 1
for j in range(5):
  A[20 + j][29 + j+38] = -1
  A[21 + j][29 + j+38] = 1
  A[22+j][29+j+38] = 1
  

#-----------------------
#and so we have

moves = A
#m = nb of moves
m = 76
#n = nb of holes
n = 33

We will call *L* the number of moves necessary to finish the puzzle. 

We will call *m* = 76 the number of possible moves (which are listed in the matrix *A*).

The solution to the problem is in the form of (*x1...xl*), and every *xi* is a unit vector of dimension *m* (*xi* [*j*] = 1 iff the move number *i*  in the winning sequence is the move number *j* in *A*)

In [0]:
def calculate_L(ps,pf):
  return(int(np.sum(ps-pf)))

G and h will contain the conditions which correspond to the following rules : 
* it is forbidden to take a peg from a hole which does not contain one (* 0 <= ps - Ax*)
* it is forbidden to place a peg on top of another (*ps-Ax <= 1*)

In other words, the intermediary positions must be made up of vectors which contain only 1s and 0s

The matrix G is 2 112 : 2 432

In [0]:
def generate_G_and_h(ps, pf) : 
  L = calculate_L(ps, pf)
  G = np.zeros((2*L*n,m*L))
  h = np.zeros(n*L*2)
  #this is to create a lower block triangle matrix, with each block being moves
  for i in range(L):
    for j in range(i+1):
      G[i*n:(i+1)*n, m*j : m*(j+1)] = moves
  #we do so twice (once for 0< and again for <1)
  for i in range(L):
    for j in range(i+1):
      G[(i+L)*n:(i+L+1)*n, m*j : m*(j+1)] = -1*moves
  #Ax <= ps
  for i in range(L):
    h[i*n : (i+1)*n] = ps
  #-Ax <= 1-ps
  for i in range(L, 2*L):
    h[i*n : (i+1)*n] = np.ones(n) - ps
  return(G, h)



A and b will contain the constraints that correspond to the following rules : 
* exacly one move is played each turn
* the starting position is ps and the end position is pf

The matrix A is 64 : 2432

In [0]:
def generate_A_and_b(ps, pf):
  #ps and pf must be np.arrays
  L = calculate_L(ps, pf)
  A = np.zeros((L + n, m*L))
  b = np.zeros(L+n)
  #those lines correspond to the condition there is exacly one move per turn
  for i in range(L):

    A[i,m*i : m*(i+1)] = np.ones(m)
    
  b[:L] = np.ones(L)
  #those lines correspond to the condition that we start in position ps and end in pf
  for i in range(L):
    A[L:L+n, m*i : m*(i+1)] = moves
  b[L:] = (ps-pf)
  return(A, b)

Because we are only interested in the existence of a solution, the cost is irrelevant, and we set it to 1

In [0]:
def generate_c(ps, pf):
  L = calculate_L(ps, pf)
  return(np.ones(m*L))

In [0]:
def solve_solitaire(ps, pf):
  L = calculate_L(ps, pf)
  G , h= generate_G_and_h(ps, pf)
  A , b= generate_A_and_b(ps, pf)
  c = generate_c(ps, pf)
    
  A = cv.matrix(A)
  G = cv.matrix(G)
  h = cv.matrix(h)
  b = cv.matrix(b)
  c = cv.matrix(c)
    
  I = set([])
  B = set(range(m*L))
  return(cvxopt.glpk.ilp(c, G, h, A, b, I, B))

#### Now we test the algorithm on a few easy cases

For a one move problem, the solution  is found

In [0]:
ps = np.ones(33)
ps[16] = 0
pf = np.ones(33)
pf[18] = 0
pf[17] = 0

In [10]:
#here we print the moves : it shows three of holes : the one we take the peg from, the one it jumps over, and where it lands (on its own line)
a = solve_solitaire(ps, pf)
print(a)
a = a[1]
solution = [i%76 for i in range(len(a)) if a[i] == 1.0]
for j in solution:
  print([i for i in range(len(moves)) if moves[i,j] == 1.0])
  print([i for i in range(len(moves)) if moves[i,j] == -1.0])

('optimal', <76x1 matrix, tc='d'>)
[17, 18]
[16]


For a 2 move problem it works as well

In [0]:
ps = np.ones(33)
ps[16] = 0
pf = np.ones(33)
pf[27] = 0
pf[22] = 0
pf[14]= 0

In [12]:
#here we print the moves : it shows three of holes : the one we take the peg from, the one it jumps over, and where it lands (on its own line)
a = solve_solitaire(ps, pf)
print(a)
a = a[1]
solution = [i%76 for i in range(len(a)) if a[i] == 1.0]
for j in solution:
  print([i for i in range(len(moves)) if moves[i,j] == 1.0])
  print([i for i in range(len(moves)) if moves[i,j] == -1.0])

('optimal', <152x1 matrix, tc='d'>)
[14, 15]
[16]
[22, 27]
[15]


For a 4 moves

In [0]:
ps = np.ones(33)
ps[16] = 0
pf = np.ones(33)
pf[3] = 0
pf[8] = 0
pf[16]= 0
pf[14]= 0
pf[13]= 0

In [14]:
#here we print the moves : it shows three of holes : the one we take the peg from, the one it jumps over, and where it lands (on its own line)
a = solve_solitaire(ps, pf)
print(a)
a = a[1]
solution = [i%76 for i in range(len(a)) if a[i] == 1.0]
for j in solution:
  print([i for i in range(len(moves)) if moves[i,j] == 1.0])
  print([i for i in range(len(moves)) if moves[i,j] == -1.0])

('optimal', <304x1 matrix, tc='d'>)
[14, 15]
[16]
[3, 8]
[15]
[15, 16]
[14]
[13, 14]
[15]


for a few more moves

In [0]:
ps = np.ones(33)
ps[16] = 0
pf = np.ones(33)
pf[0 : 6] = np.zeros(6)
pf[8] = 0
pf[9]= 0
pf[10]= 0
pf[16]= 0

In [16]:
#here we print the moves : it shows three of holes : the one we take the peg from, the one it jumps over, and where it lands (on its own line)
a = solve_solitaire(ps, pf)
print(a)
a = a[1]
solution = [i%76 for i in range(len(a)) if a[i] == 1.0]
for j in solution:
  print([i for i in range(len(moves)) if moves[i,j] == 1.0])
  print([i for i in range(len(moves)) if moves[i,j] == -1.0])

('optimal', <684x1 matrix, tc='d'>)
[4, 9]
[16]
[10, 11]
[9]
[9, 16]
[4]
[7, 8]
[9]
[2, 5]
[10]
[0, 3]
[8]
[9, 10]
[11]
[1, 4]
[9]
[8, 9]
[7]


#### Now we test the algorithm with the original problem

As expected, it takes too much time

In [0]:
ps = np.ones(33)
ps[16] = 0
pf = np.zeros(33)
pf[16] = 1

In [0]:
solve_solitaire(ps, pf)

### Improving performance using Pagoda functions.

In an effort to reduce the size of the branch-and-bound tree, we try to add the constraint that every step's Pagoda value must be higher than *pf*'s.

Adding a Pagoda function constraint only requires adding n lines (one for every turn) to G and h, but it does not reduce the tree by a large amount. This is why we wanted to try adding several.

In [0]:
def generate_G_and_h_with_pagoda(pagodas, ps, pf) : 
  nb_pagodas = len(pagodas)
  L = calculate_L(ps, pf)
  G = np.zeros((2*L*n + nb_pagodas*n,m*L))
  h = np.zeros(n*L*2+nb_pagodas*n)
  #this is to create a lower block triangle matrix, with each block being moves
  for i in range(L):
    for j in range(i+1):
      G[i*n:(i+1)*n, m*j : m*(j+1)] = moves
  #we do so twice (once for 0< and again for <1)
  for i in range(L):
    for j in range(i+1):
      G[(i+L)*n:(i+L+1)*n, m*j : m*(j+1)] = -1*moves
  #Ax <= ps
  for i in range(L):
    h[i*n : (i+1)*n] = ps
  #-Ax <= 1-ps
  for i in range(L, 2*L):
    h[i*n : (i+1)*n] = np.ones(n) - ps
    
  #this is to create the pagoda constraints
  for k in range(len(pagodas)):
    dot_product = np.dot(pagodas[k] , moves)
    for i in range(L-1):
      for j in range(i+1):
        G[(2*L + k*L + i), m*j : m*(j+1)] = 1*dot_product-np.dot(pagodas[k], ps)
    for i in range(L-1):
      h[2*L + k*L + i] = 1*np.dot(pagodas[k], pf)-np.dot(pagodas[k], ps)
  return(G, h)

In [0]:
def solve_solitaire_with_pagoda(pagodas, ps, pf):
  L = calculate_L(ps, pf)
  G , h= generate_G_and_h_with_pagoda(pagodas, ps, pf)
  A , b= generate_A_and_b(ps, pf)
  c = generate_c(ps, pf)
  
  
  A = cv.matrix(A)
  G = cv.matrix(G)
  h = cv.matrix(h)
  b = cv.matrix(b)
  c = cv.matrix(c)
    
  I = set([])
  B = set(range(m*L))
  return(cvxopt.glpk.ilp(c, G, h, A, b, I, B))

We check that with an empty list of pagodas, it works the same as before

In [0]:
ps = np.ones(33)
ps[16] = 0
pf = np.ones(33)
pf[27] = 0
pf[22] = 0
pf[14]= 0

In [22]:
#here we print the moves : it shows three of holes : the one we take the peg from, the one it jumps over, and where it lands (on its own line)
a = solve_solitaire_with_pagoda([], ps, pf)
print(a)
a = a[1]
solution = [i%76 for i in range(len(a)) if a[i] == 1.0]
for j in solution:
  print([i for i in range(len(moves)) if moves[i,j] == 1.0])
  print([i for i in range(len(moves)) if moves[i,j] == -1.0])

('optimal', <152x1 matrix, tc='d'>)
[14, 15]
[16]
[22, 27]
[15]


It does.

Now we try adding pagodas

In [0]:
#those are the pagodas found in both papers
pagoda1 = [-.3,.4,0.,1.,0.,1.,.5,0.,.5,.4,.1,.3,-.1,0.,.9,.7,.3,.9,1.1,.4,.5,.6,.1,.5,.2,.6,.2,.8,0.,.8,0.,.5,-.2]
pagoda2 = [0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.]
pagoda3 = [0.,1.,0.,0.,0.,0.,0.,1.,0.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,1.,0.,1.,0.,0.,0.,0.,0.,1.,0.]
pagoda4 = [1.,0.,1.,0.,0.,0.,1.,0.,1.,0.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,1.,0.,1.,0.,1.,0.,1.,0.,0.,0.,1.,0.,1.]
pagoda5 = [ 0, 0,  0, 0, 1, 0, -1, 1, 0, 1, 0, 1, -1 , 0,  0, 0, 0, 0, 0,  0 ,-1, 1, 0, 1, 0, 1, -1,0, 1, 0 ,0, 0, 0 ]
pagoda6 = [  -1, 0, -1,1, 0, 1,0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,  0, 0, 0, 0, 0, 0, 0, 1, 0, 1, -1, 0, -1]
pagoda7 = [ 0, 0, 0,1, 0, 1, 0,  0, 0, 0, 0, 0,  0 ,1,  0, 1, 0, 1, 0,  1, 0,  0, 0, 0, 0, 0,  0,1, 0, 1,0, 0, 0,]


pagodas =[pagoda1, pagoda2, pagoda3, pagoda4]


In [0]:
ps = np.ones(33)
ps[16] = 0
pf = np.ones(33)
pf[27] = 0
pf[22] = 0
pf[14]= 0

In [25]:
a = solve_solitaire_with_pagoda(pagodas, ps, pf)
print(a)
a = a[1]
solution = [i%76 for i in range(len(a)) if a[i] == 1.0]
for j in solution:
  print([i for i in range(len(moves)) if moves[i,j] == 1.0])
  print([i for i in range(len(moves)) if moves[i,j] == -1.0])

('optimal', <152x1 matrix, tc='d'>)
[14, 15]
[16]
[22, 27]
[15]


now we try to compare the performance of the solvers with an without the pagoda constraints

In [0]:
ps = np.ones(33)
ps[16] = 0
pf = np.ones(33)
pf[0 : 6] = np.zeros(6)
pf[8] =0 
pf[9]= 0

pf[11] = 0
pf[15] = 0
pf[16]= 0
pf[24] = 0 
pf[23] = 0
pf[29] = 0

In [27]:
#here we print the moves : it shows three of holes : the one we take the peg from, the one it jumps over, and where it lands (on its own line)
t = time.time()
a = solve_solitaire_with_pagoda(pagodas, ps, pf)
t_prime = time.time()
print(t_prime-t)
#print(a)
#a = a[1]
#solution = [i%76 for i in range(len(a)) if a[i] == 1.0]
#for j in solution:
#  print([i for i in range(len(moves)) if moves[i,j] == 1.0])
#  print([i for i in range(len(moves)) if moves[i,j] == -1.0])

4.50611114502e-05


In [28]:
#here we print the moves : it shows three of holes : the one we take the peg from, the one it jumps over, and where it lands (on its own line)
t = time.time()
a = solve_solitaire(ps, pf)
t_prime = time.time()
print(t_prime-t)
#print(a)
#a = a[1]
#solution = [i%76 for i in range(len(a)) if a[i] == 1.0]
#for j in solution:
#  print([i for i in range(len(moves)) if moves[i,j] == 1.0])
#  print([i for i in range(len(moves)) if moves[i,j] == -1.0])

5.60283660889e-05


We can see that with the pagodas, the search finishes faster, the problem we have tried finishes in 45s with the pagoda functions, and didn't finish at all without them. **Maybe if we had even more pagodas, it would be even faster, as that would help decrease the solution space in which the algorithm does the backtracking. **

We have found that adding pagoda functions can, contrarywise, increase the time of executions, perhaps because adding constraints makes each recursive call longer?

In [0]:
ps = np.ones(33)
ps[16] = 0
pf = np.zeros(33)
pf[16] = 1

In [0]:
#here we print the moves : it shows three of holes : the one we take the peg from, the one it jumps over, and where it lands (on its own line)
t = time.time()
a = solve_solitaire_with_pagoda(pagodas, ps, pf)
t_prime = time.time()
print(t_prime-t)
print(a)
a = a[1]
solution = [i%76 for i in range(len(a)) if a[i] == 1.0]
for j in solution:
  print([i for i in range(len(moves)) if moves[i,j] == 1.0])
  print([i for i in range(len(moves)) if moves[i,j] == -1.0])

## Conclusion

Model A is relatively easy to understand and provides a good first approach to the problem. However, in its naive version, the algorithm takes too long to run. 

The second model we studied has the advantage of presenting less variables. Pagodas functions were able to help speed up the process, but it still seems to take too long.